In [4]:
import numpy as np
import pandas as pd

import bokeh.layouts
from bokeh.io import curdoc, output_file, show
from bokeh.layouts import widgetbox
from bokeh.models import (
    BoxSelectTool,
    Circle, 
    ColumnDataSource,
    CustomJS,
    DataRange1d, 
    GMapPlot,
    GMapOptions,
    HoverTool,
    LogColorMapper,
    PanTool,
    Slider,
    WheelZoomTool,
)
from bokeh.models.glyphs import Patches
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

In [5]:
with open('NYC_boundries_cleaned.json') as f:
    data = json.load(f)
nyc_neighborhoods = data
len(nyc_neighborhoods)

195

# 3. Build Bokeh heat map

In [6]:
palette.reverse()

color_mapper = LogColorMapper(palette=palette)

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]

neighborhood_names = list(nyc_neighborhoods.keys())

# list 195 columns
# 

neighborhood_counts = np.zeros(len(nyc_neighborhoods), dtype=int)

# All 195 columns with given input value
months = np.zeros(len(nyc_neighborhoods), dtype=int)

days = np.zeros(len(nyc_neighborhoods), dtype=int)

hours = np.zeros(len(nyc_neighborhoods), dtype=int)


source = ColumnDataSource(data=dict(
    x = neighborhood_xs,
    y = neighborhood_ys,
    name = neighborhood_names,
    month = months,
    day = days,
    hour = hours,
    value = neighborhood_counts
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

plot = figure(title="Uber Pickup Distribution", tools=TOOLS, 
              x_axis_location=None, y_axis_location=None)

plot.grid.grid_line_color = None

plot.patches('x', 'y', source=source,
              fill_color={'field': 'value', 'transform': color_mapper},
              fill_alpha=0.7, line_color="black", line_width=0.5)

hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("(Long, Lat)", "($x, $y)"),
    ("Number of pickups", "@value"),
    ("Month", "@month"),
    ("Day", "@day"),
    ("Hour", "@hour")
]

In [7]:
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    
    var month = month.value;
    var day = day.value;
    var hour = hour.value;
    
    for (i = 0; i < data['month'].length; i++) {
        data['month'][i] = month
    }
    
     for (i = 0; i < data['day'].length; i++) {
        data['day'][i] = day
    }
    
     for (i = 0; i < data['hour'].length; i++) {
        data['hour'][i] = hour
    }
    
    for (i = 0; i < data['value'].length; i++) {
        data['value'][i] = hour
    }
    
    source.trigger('change');
""")




In [34]:
checkbox_button_group = CheckboxButtonGroup(labels=["Uber", "Taxi"], active=[0, 1])

hour_slider = Slider(start=0, end=23, step=1, value=0, title="Hour", callback = callback)
callback.args["hour"] = hour_slider

month_select = Select(title="Month:", value="Month", options=['4', '5', '6'], callback=callback)
callback.args["month"] = month_select
                                        
days = [str(i) for i in list(range(1,32))]
day_select = Select(title="Day:", value="Day", options=days, callback=callback)
callback.args["day"] = day_select

In [35]:
curdoc().clear()
show(bokeh.layouts.row(plot, checkbox_button_group, widgetbox(hour_slider, month_select, day_select, width=300)))